In [47]:
import csv
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zawar_keys import tapi_key,oapi_key

In [26]:
url='https://api.themoviedb.org/3/movie/top_rated?api_key={}'.format(tapi_key)
url_params={'page': 1}

In [28]:
def get_tmdb_toprated(params,api_key):
    url='https://api.themoviedb.org/3/movie/top_rated?api_key={}'.format(api_key)
    response=requests.get(url,params=params)
    return response.json()

In [29]:
response=get_tmdb_toprated(url_params,tapi_key)

In [57]:
response['total_pages']

357

In [44]:
def parse_toprated(toprated):
    parsed_data=[]
    for i in toprated['results']:
        data=[
            i['id'],i['title'],i['original_language'],i['vote_average'],i['vote_count'],i['popularity'],
            i['genre_ids'],i['release_date'],i['adult']
        ]
        parsed_data.append(data)
    return parsed_data

In [64]:
def save_data(file_name,data):
    with open(file_name, "a", newline="", encoding='utf-8') as a:
        writer = csv.writer(a)
        writer.writerows(data)

In [66]:
with open("TMDB_toprated.csv", "w", newline="", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows([['id','title','original_language','avg_vote','vote_count','popularity','genres','release_date','adult']])

In [67]:
p=1
while p<=357:
    url_params['page']=p
    
    results=get_tmdb_toprated(url_params,tapi_key)
    
    parsed_results=parse_toprated(results)
    
    save_data("TMDB_toprated.csv",parsed_results)
    p+=1

In [68]:
print(p)

358


In [72]:
toprated_df=pd.read_csv('TMDB_toprated.csv')
mov_ids=toprated_df['id'].values
mov_ids

array([19404,   278,   238, ...,  5491, 11059, 14164], dtype=int64)

In [70]:
def get_movie_details(mov_id,api_key):
    url2='https://api.themoviedb.org/3/movie/{}?api_key={}'.format(mov_id,api_key)
    response=requests.get(url2)
    return response.json()

In [76]:
def parse_movie(mov_data):
    parsed_data=[
        mov_data['id'],mov_data['imdb_id'],mov_data['budget'],mov_data['revenue'],
        [genre['name'] for genre in mov_data['genres']],mov_data['original_title'],
        [c['name'] for c in mov_data['production_countries']],mov_data['runtime'],mov_data['status']
    ]
    return parsed_data

In [78]:
with open("TMDB_movie_details.csv", "w", newline="", encoding='utf-8') as m:
    writer = csv.writer(m)
    writer.writerows([['id','imdb_id','budget','revenue','genre_names','original_title','production_countries',
                       'runtime','status']])

In [88]:
for mov_id in mov_ids[4996:]:
    mov_info=get_movie_details(mov_id,tapi_key)
    
    parsed_mov=[parse_movie(mov_info)]
    
    save_data("TMDB_movie_details.csv", parsed_mov)

In [115]:
df2=pd.read_csv('TMDB_movie_details.csv')
countries=df2['production_countries'].values

In [143]:
import ast
type(ast.literal_eval(df2['production_countries'].values[0]))

list

In [147]:
df2['USA']=np.where('United States of America' in df2['countries'],1,0)

In [145]:
df2['countries']=df2['production_countries'].apply(lambda x: ast.literal_eval(x))

In [155]:
df2['USA']=df2['countries'].apply(lambda x: 'United States of America' in x)

In [163]:
df2[['id','imdb_id','countries','USA']].head(15)

,id,imdb_id,countries,USA
0,19404,tt0112870,[India],False
1,278,tt0111161,[United States of America],True
2,238,tt0068646,[United States of America],True
3,424,tt0108052,[United States of America],True
4,42269,tt0075793,[Italy],False
5,240,tt0071562,[United States of America],True
6,372058,tt5311514,[Japan],False
7,496243,tt6751668,[South Korea],False
8,129,tt0245429,[Japan],False
9,497,tt0120689,[United States of America],True


In [159]:
df2[df2['USA']==True].shape

(5332, 11)